In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
cd /home/juliakhalina/notebooks

/home/juliakhalina/notebooks


In [3]:
pwd

'/home/juliakhalina/notebooks'

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 21:31:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv')

In [6]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [7]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [8]:
import pandas as pd

In [9]:
df_pandas = pd.read_csv('fhvhv_tripdata_2021-06.csv')

In [11]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [11]:
df

DataFrame[dispatching_base_num: string, pickup_datetime: string, dropoff_datetime: string, PULocationID: string, DOLocationID: string, SR_Flag: string, Affiliated_base_number: string]

In [13]:
df.select('pickup_datetime', 'dropoff_datetime').show()

+-------------------+-------------------+
|    pickup_datetime|   dropoff_datetime|
+-------------------+-------------------+
|2021-06-01 00:02:41|2021-06-01 00:07:46|
|2021-06-01 00:16:16|2021-06-01 00:21:14|
|2021-06-01 00:27:01|2021-06-01 00:42:11|
|2021-06-01 00:46:08|2021-06-01 00:53:45|
|2021-06-01 00:45:42|2021-06-01 01:03:33|
|2021-06-01 00:18:15|2021-06-01 00:25:47|
|2021-06-01 00:33:06|2021-06-01 00:42:46|
|2021-06-01 00:46:27|2021-06-01 00:56:50|
|2021-06-01 00:48:06|2021-06-01 01:04:10|
|2021-06-01 00:18:54|2021-06-01 00:26:14|
|2021-06-01 00:31:02|2021-06-01 00:36:39|
|2021-06-01 00:41:53|2021-06-01 01:07:32|
|2021-06-01 00:29:52|2021-06-01 00:54:41|
|2021-06-01 00:15:57|2021-06-01 00:39:36|
|2021-06-01 00:11:59|2021-06-01 00:23:32|
|2021-06-01 00:30:35|2021-06-01 00:45:35|
|2021-06-01 00:49:01|2021-06-01 01:03:50|
|2021-06-01 00:07:36|2021-06-01 00:21:13|
|2021-06-01 00:25:48|2021-06-01 00:40:43|
|2021-06-01 00:46:11|2021-06-01 00:53:39|
+-------------------+-------------

In [14]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [15]:
from pyspark.sql import types

In [16]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.IntegerType(), True)
    ])

In [17]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv')

In [18]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: integer (nullable = true)



In [27]:
from pyspark.sql import functions as F

In [31]:
df = df.withColumn('date_diff', F.datediff(F.to_date(df.pickup_datetime), F.to_date(df.dropoff_datetime)))
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|date_diff|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                  null|        0|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                  null|        0|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                  null|        0|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                  null|        0|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|  

In [43]:
dfs_1 = df.withColumn('hours_dif', (df.date_diff*24) + F.hour(df.pickup_datetime) - F.hour(df.dropoff_datetime))
dfs_1.sort('hours_dif').show(truncate=False)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+---------+
|dispatching_base_num|pickup_datetime    |dropoff_datetime   |PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|date_diff|hours_dif|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+---------+---------+
|B02872              |2021-06-25 13:55:41|2021-06-28 08:48:25|98          |265         |N      |null                  |-3       |-67      |
|B02765              |2021-06-22 12:09:45|2021-06-23 13:42:44|188         |198         |N      |null                  |-1       |-25      |
|B02879              |2021-06-27 10:32:29|2021-06-28 06:31:20|78          |169         |N      |null                  |-1       |-20      |
|B02800              |2021-06-26 22:37:11|2021-06-27 16:49:01|263         |36          |N      |null                  |-1       |-18      |
|B02682             